In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#import class 
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "cat"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
                               





In [20]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#logo for dashboard
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#HTML image tag in the line below into the app.layout code according to your design
#include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    
    #layout for title
    html.Center(html.B(html.H1(children='Zuzana Martins', 
       style={'textAlign': 'center', 
          'color': '#6EA2D6'}))),
    html.A([
        
        #layout for image
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'textAlign': 'center',
                'height' : '20%',
                'width' : '20%',
            })
        
    #image is clickable
    ], href='https://www.snhu.edu'),        
            
    html.Hr(),
    html.Div(
        
        #radio items for easy filtering
        dcc.RadioItems( 
            id='filter-type',
            options=[
                
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOWR'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DROIT'},
                {'label': 'Reset', 'value': 'Reset'}
                    ],
            
            value='Reset',
            labelStyle={'display': 'inline-block'}
                    )  
            ),
    
    #table set up including interactive features
    html.Hr(),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        hidden_columns=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    
    #This sets up the dashboard so that chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

app

In [25]:
#############################################
# Interaction Between Components / Controller
#############################################
  

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        #adjusts the read request for the desired dog type and status
        if filter_type == 'WR':
            
            df = pd.DataFrame(list(shelter.read( { "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gte" : 26 },
                                                  "sex_upon_outcome" : "Intact Female",
                                                  "breed" : { "$in" : [
                                                  "Labrador Retriever Mix",
                                                  "Chesa Bay Retr Mix",
                                                  "Newfoundland Mix" ] }
                                                   } ).sort({"age_upon_outcome_in_weeks" : 1} )
                                  ))
            
        #adjusts the read request for the desired dog type and status
        elif filter_type == 'MWR':
            # breeds and ages determined by Grazioso
            df = pd.DataFrame(list(shelter.read( { "age_upon_outcome_in_weeks" : { "$lt" : 156, "$gte" : 26 },
                                                  "sex_upon_outcome" : "Intact Male",
                                                  "breed" : { "$in" : [
                                                  "German Shepherd Mix",
                                                  "Alaskan Malamute",
                                                  "Siberian Husky Mix",
                                                  "Rottweiler Mix",
                                                  "Old English Sheepdog" ] }
                                                   } ).sort({"age_upon_outcome_in_weeks" : 1} ) 
                                 ))
            
        #adjusts the read request for the desired dog type and status
        elif filter_type == 'DROIT':
            #breeds and ages determined by Grazioso
            df = pd.DataFrame(list(shelter.read( { "age_upon_outcome_in_weeks" : { "$lt" : 300, "$gte" : 26 },
                                                  "sex_upon_outcome" : "Intact Male",
                                                  "breed" : { "$in" : [
                                                  "German Shepherd Mix",
                                                  "Golden Retriever Mix",
                                                  "Bloodhound Mix",
                                                  "Rottweiler Mix",
                                                  "Doberman Pinsch" ] }
                                               } ).sort({"age_upon_outcome_in_weeks" : 1} ) 
                                   
                                  ))
            
        #resets the search to nothing to allow all results to be displayed
        elif filter_type == 'Reset':
            df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)



#style updates for the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#graph/pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, values=values, names=names, color_discrete_sequence=px.colors.sequential.RdBu)
        )    
    ]

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_rows')]
)
def update_styles(selected_rows):
    """ highlight selected rows of dash table in blue """
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]


@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-interactivity', "derived_virtual_data")])

def update_graph(allData):
    """ functionality for pie chart """
    dff = pd.DataFrame(allData)

    piechart = px.pie(
        data_frame=dff,
        # pie chart measures outcome_type column
        names=dff['outcome_type'],
        hole=.3,
    )
    return piechart


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', 'derived_virtual_selected_rows')
     ])
def update_map(viewData, derived_virtual_selected_rows):
    """ functionality for updating map. map always shows location of the animal at top of table's current page """

    dff = df if viewData is None else pd.DataFrame(viewData)
    selected_animal = None

    # if there are no selected rows yet, map default displays first animal of table's current page
    if not derived_virtual_selected_rows:
        selected_animal = dff.iloc[0]
    # else there is a selected row, map displays that animal
    else:
        selected_animal = dff.iloc[derived_virtual_selected_rows[0]]

    latitude = selected_animal[12]
    longitude = selected_animal[13]
    animal_breed = selected_animal[3]
    animal_name = selected_animal[8]

    return [
        dl.Map(style={'width': '700px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[latitude, longitude], children=[
                # show breed of animal on hovering over marker
                dl.Tooltip(animal_breed),
                # show animal name on clicking marker
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)
                ])
            ])
        ])
    ]



app